<a href="https://colab.research.google.com/github/vifirsanova/compling/blob/main/attention_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`Курс "Компьютерная лингвистика" | НИУ ВШЭ Санкт-Петербург 2024 (c) В.И. Фирсанова`

# Эксперимент [Bahdanau et al. (2014)](https://arxiv.org/abs/1409.0473)

## Теория

**RNN и проблема длинных предложений**

 Вектор фиксированной длины размера окна n ->

 -> не можем обработать длинные последовательности,

 -> теряем информацию о рекурсивных конструкциях естественного языка,

 -> градиент взрывается или исчезает

Решение: **механизм внимания**. Первоначально итеративный:

1. Оценка выравнивания `a(s_t-1, h_i)`, где
  1. `h_i` - скрытое состояние кодера (текущее состояние векторного пространства эмбеддингов, которое построила наша модель для входного текста; например, эмбеддинг для `меня зовут Джон`),
  2. `s_t-1` - выходные данные декодера на предыдущем шаге вычислений `t-1` (то, что модель расшифровала на основе предыдущего скрытого состояния; например, декодер расшифровал вектор `меня зовут` и получилось `my name is`),
  3. `a` - это оценка связи между *текущими* входными и *текущими* выходными данными на шаге `t` (итак, нам нужна нейросеть, которая расчитает `a("my name is", "меня зовут Джон")`, чтобы оценить силу связи между декодированными выходными данными `John` и векторным представлением `меня зовут Иван` на основе предыдущего шага декодера `t-1`). Чем выше связь, тем выше оценка.
2. Текущая оценка `a` (для слова `John`) преобразовывается к виду весов `w_a`, чтобы их можно было использовать в качестве параметра. Применяем `softmax`, чтобы преобразовать оценки к виду коэффициентов от 0 до 1. Каждое слово в нашем словаре получит такой коэффициент.
3. Вектор контекста `c_t` - это взвешенная сумма всех скрытых состояний кодера `v("меня" * w_a_t-2) + v("меня зовут" * w_a_t-1) + v("меня зовут Джон" * w_a_t)`; этот вектор мы будем скармливать декодеру на текущем шаге `t`, чтобы получился вектор внимания для слова `John`.

Как это выглядит **в коде**? В коде мы сразу создаем матрицу внимания для всего датасета, без необходимости в последовательном, итерационном расчете с помощью трех матриц: `queries`, `keys`, `values`.

1. Оценка выравнивания: `k_i*q`, где
  1. `k_i` - база данных ключей (keys), список закодированных входных токенов (`[v"меня", v"зовут", v"Джон"]`),
  2. `q` - вектор текущего шага, для которого мы строим представление с помощью механизма внимания, наш `John`. На этом месте последовательно окажется каждый из векторов нашего декодера: `q_1: my`; `q_2: name`; `q_3: is`; `q_4: John`. При этом база `k_i` меняться не будет. Таким образом, у нас получится 4 матрицы `a_my`, `a_name`, `a_is`, `a_John`, в каждой из которых хранится оценка связи между `q_n` и каждым из элементов `k_i`. Например: `{John: меня, 12}; {John: зовут, 10}; {John: Джон, 78}`.
2. `a = softmax(k_i*q)`, преобразовали все оценки к виду коэффициентов от 0 до 1.
3. `a*v_k`, где `v_k` - вектор ключа (совпадает с `k_i`, но для удобства расчетов хранится в отдельной базе данных); итак, получается вектор внимания - взвешенная сумма `attention_John = ({v"меня" * 0.2} + {v"зовут" * 0.2}; {v"Джон" * 0.6})`.

**Итого**: каждое слово в нашем корпусе Y представлено его силой связи с *каждым* из слова корпуса X.


## Механизм внимания с нуля с помощью NumPy

In [ ]:
# датасет

source_sentences = ["My name is John", "His name is John", "Her name is Jane", "My name is Jane"]
target_sentences = ["Меня зовут Джон", "Его зовут Джон", "Ее зовут Джейн", "Меня зовут Джейн"]

In [ ]:
# токенизация

import re

def tokenize(sentence):
    tokens = sentence.lower().split()
    return [re.sub(r'[^\w\s]', '', token) for token in tokens]

source_tokens, target_tokens = [tokenize(sent) for sent in source_sentences], [tokenize(sent) for sent in target_sentences]

print("Исходные токены:\n", source_tokens)
print()
print("Целевые токены:\n", target_tokens)

Исходные токены:
 [['my', 'name', 'is', 'john'], ['his', 'name', 'is', 'john'], ['her', 'name', 'is', 'jane'], ['my', 'name', 'is', 'jane']]

Целевые токены:
 [['меня', 'зовут', 'джон'], ['его', 'зовут', 'джон'], ['ее', 'зовут', 'джейн'], ['меня', 'зовут', 'джейн']]


In [ ]:
# мешок слов

from collections import Counter
import numpy as np

def bag_of_words(tokenized_sentences):

  print("Корпус:\n", tokenized_sentences)

  # словарь уникальных словоформ
  words = [word for sentence in tokenized_sentences for word in sentence]
  unique_words = sorted(set(words))
  print("\nУникальные словоформы корпуса:\n", unique_words)

  print("\nМешок слов:")
  matrix = []

  for word in unique_words:
    temp = []
    for sent in tokenized_sentences:
      temp.append(1 if word in sent else 0)
    matrix.append(temp)
    print({word: temp})

  return np.array(matrix)

bow_source, bow_target = bag_of_words(source_tokens), bag_of_words(target_tokens)

print("\nРезультат для целевого корпуса (K, V):\n", bow_source)
print("\nРезультат для исходного корпуса (Q):\n", bow_target)

Корпус:
 [['my', 'name', 'is', 'john'], ['his', 'name', 'is', 'john'], ['her', 'name', 'is', 'jane'], ['my', 'name', 'is', 'jane']]

Уникальные словоформы корпуса:
 ['her', 'his', 'is', 'jane', 'john', 'my', 'name']

Мешок слов:
{'her': [0, 0, 1, 0]}
{'his': [0, 1, 0, 0]}
{'is': [1, 1, 1, 1]}
{'jane': [0, 0, 1, 1]}
{'john': [1, 1, 0, 0]}
{'my': [1, 0, 0, 1]}
{'name': [1, 1, 1, 1]}
Корпус:
 [['меня', 'зовут', 'джон'], ['его', 'зовут', 'джон'], ['ее', 'зовут', 'джейн'], ['меня', 'зовут', 'джейн']]

Уникальные словоформы корпуса:
 ['джейн', 'джон', 'его', 'ее', 'зовут', 'меня']

Мешок слов:
{'джейн': [0, 0, 1, 1]}
{'джон': [1, 1, 0, 0]}
{'его': [0, 1, 0, 0]}
{'ее': [0, 0, 1, 0]}
{'зовут': [1, 1, 1, 1]}
{'меня': [1, 0, 0, 1]}

Результат для целевого корпуса (K, V):
 [[0 0 1 0]
 [0 1 0 0]
 [1 1 1 1]
 [0 0 1 1]
 [1 1 0 0]
 [1 0 0 1]
 [1 1 1 1]]

Результат для исходного корпуса (Q):
 [[0 0 1 1]
 [1 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [1 1 1 1]
 [1 0 0 1]]


In [ ]:
# случайные веса (в идеале, в этой ячейке мы используем метод обратного распространения ошибки для расчета весов)
# 4 - размерность вектора, равна количеству текстов в параллельном корпусе (здесь!)

from numpy import random

random.seed(2024)

W_Q = random.randint(2, size=(4, 4))
W_K = random.randint(2, size=(4, 4))
W_V = random.randint(2, size=(4, 4))

W_Q

array([[0, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 1, 1, 0],
       [1, 0, 0, 0]])

**Метод обратного распространения ошибки (backpropagation)** - метод вычисления градиента для обновления весов нейронной сети.

1. forward pass / propagation: входные данные передаются нейронной сети, чтобы получить предсказания на выходе
2. backward pass: вычисляем градиент функции потерь и обновляем веса нейросети

Псевдокод процесса обучения:

```
for epoch in epochs:    
    
    1. feedforward propagation
    # скрытый слой
    result_n-1 = func(x_train, weight_n-1)
    activated_n-1 = activation_func(result_n-1)

    # выходной слой
    result_n = func(activated_n-1, weight_n)
    activated_n = activation_func(result_n)
    
    2. вычисление ошибки
    mean_squared_error(activated_n, y_train)
    metric_score(activated_n, y_train)
    
    3. backpropagation
    delta_weight_n-1
    delta_weight_n
    
    4. обновление весов
    learning_rate * weight_n_update
    learning_rate * weight_n-1_update
```

In [ ]:
# умножение матриц: взвешиваем каждый эмбеддинг

query_1 = bow_target[0] @ W_Q
key_1 = bow_source[0] @ W_K
value_1 = bow_source[0] @ W_V

query_1

array([2, 1, 1, 0])

In [ ]:
# матричные операции позволяют делать это не итеративно, но для корпуса целиком

Q = bow_target @ W_Q
K = bow_source @ W_K
V = bow_source @ W_V

Q

array([[2, 1, 1, 0],
       [1, 0, 0, 1],
       [1, 0, 0, 1],
       [1, 1, 1, 0],
       [3, 1, 1, 1],
       [1, 0, 0, 0]])

In [ ]:
# теперь можно получить скор выравнивания

from numpy import dot

scores = np.array([dot(Q[0], K[0]), dot(Q[0], K[1]), dot(Q[0], K[1]), dot(Q[0], K[2])])

scores

array([ 4,  3,  3, 13])

In [ ]:
# для вычислений матрицу ключей нужно транспонировать

scores = Q @ K.transpose()

scores

array([[ 4,  3, 13,  7,  6,  6, 13],
       [ 1,  1,  4,  2,  2,  2,  4],
       [ 1,  1,  4,  2,  2,  2,  4],
       [ 3,  2,  9,  5,  4,  4,  9],
       [ 5,  4, 17,  9,  8,  8, 17],
       [ 1,  1,  4,  2,  2,  2,  4]])

**Транспонирование матрицы** - замена строк матрицы на ее столбцы, и наоборот. Иногда нам нужно транспонировать матрицу перед операцией умножения, чтобы структуры матриц были сопоставимы, а умножение возможным. Иногда транспонирование требуется, чтобы получить больше информации о том, что в ней хранится.

In [ ]:
# эти значения нужно нормализовать; применим softmax

from scipy.special import softmax

a = softmax(scores)
a

array([[1.10909001e-06, 4.08011413e-07, 8.98704943e-03, 2.22766683e-05,
        8.19512830e-06, 8.19512830e-06, 8.98704943e-03],
       [5.52183401e-08, 5.52183401e-08, 1.10909001e-06, 1.50099011e-07,
        1.50099011e-07, 1.50099011e-07, 1.10909001e-06],
       [5.52183401e-08, 5.52183401e-08, 1.10909001e-06, 1.50099011e-07,
        1.50099011e-07, 1.50099011e-07, 1.10909001e-06],
       [4.08011413e-07, 1.50099011e-07, 1.64603552e-04, 3.01481922e-06,
        1.10909001e-06, 1.10909001e-06, 1.64603552e-04],
       [3.01481922e-06, 1.10909001e-06, 4.90676273e-01, 1.64603552e-04,
        6.05542627e-05, 6.05542627e-05, 4.90676273e-01],
       [5.52183401e-08, 5.52183401e-08, 1.10909001e-06, 1.50099011e-07,
        1.50099011e-07, 1.50099011e-07, 1.10909001e-06]])

In [ ]:
# теперь мы можем вычислить внимание: взвешенную сумму значений

A = a @ V
A

array([[3.59950598e-02, 5.39925444e-02, 3.59872727e-02, 5.39847573e-02],
       [5.03675608e-06, 7.46025345e-06, 4.94187541e-06, 7.36537278e-06],
       [5.03675608e-06, 7.46025345e-06, 4.94187541e-06, 7.36537278e-06],
       [6.64756297e-04, 9.97386232e-04, 6.63797306e-04, 9.96427241e-04],
       [1.96305136e+00, 2.94457152e+00, 1.96299191e+00, 2.94451208e+00],
       [5.03675608e-06, 7.46025345e-06, 4.94187541e-06, 7.36537278e-06]])

In [ ]:
unique_words =  ['джейн', 'джон', 'его', 'ее', 'зовут', 'меня']

for i in range(len(unique_words)):
  print(unique_words[i], A[i])

джейн [0.03599506 0.05399254 0.03598727 0.05398476]
джон [5.03675608e-06 7.46025345e-06 4.94187541e-06 7.36537278e-06]
его [5.03675608e-06 7.46025345e-06 4.94187541e-06 7.36537278e-06]
ее [0.00066476 0.00099739 0.0006638  0.00099643]
зовут [1.96305136 2.94457152 1.96299191 2.94451208]
меня [5.03675608e-06 7.46025345e-06 4.94187541e-06 7.36537278e-06]


In [ ]:
print(source_tokens[0])

sorted([elem[0] for elem in A]) # джон меня его ... (результат со случайным распределением весов)

['my', 'name', 'is', 'john']


[5.036756079674562e-06,
 5.036756079674562e-06,
 5.036756079674562e-06,
 0.0006647562973232436,
 0.03599505976974601,
 1.963051358687789]

In [ ]:
# как автоматизировать поиск токенов с наибольшим весом?
np.argmax([0, 1, 3, 5])

3

In [146]:
from keras.layers import Dense, SimpleRNN, Attention
from keras.models import Sequential
import tensorflow as tf

model = Sequential()
model.add(SimpleRNN(units=3))
model.add(Dense(units=1))
model.compile()
model.build(input_shape=(2,2,2))
model.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_25 (SimpleRNN)   (2, 3)                    18        
                                                                 
 dense_19 (Dense)            (2, 1)                    4         
                                                                 
Total params: 22 (88.00 Byte)
Trainable params: 22 (88.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
